### Libraries

In [1]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

### Dataset

In [2]:
steamData = pd.read_csv('steam_data/steam.csv')
steamSupportInfo = pd.read_csv('steam_data/steam_support_info.csv')
steamChart = pd.read_csv('steam_data/steamCharts.csv')
steamSpyTagData = pd.read_csv('steam_data/steamspy_tag_data.csv')
steamGames = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_games/steam_games*.csv"))))
steamMediaData = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_media_data/steam_media_data*.csv"))))
steamRequirements = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_requirements/steam_requirements*.csv"))))
steamDescription = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_description/steam_description*.csv"))))

#game reviews
steamDayZReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/DayZ/steam_review_*.csv"))))
steamDeadbyDaylightReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/DeadbyDaylight/steam_review_*.csv"))))
steamFallout4Review = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/Fallout4/steam_review_*.csv"))))
steamNoMansSkyReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/NoMansSky/steam_review_*.csv"))))
steamPAYDAY2Review = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/PAYDAY2/steam_review_*.csv"))))
steamPLAYERUNKNOWNSBATTLEGROUNDSReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/PLAYERUNKNOWNSBATTLEGROUNDS/steam_review_*.csv"))))
steamSubnauticaReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/Subnautica/steam_review_*.csv"))))
steamTerrariaReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/Terraria/steam_review_*.csv"))))
steamTheElderScrollsVSkyrimReview = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/TheElderScrollsVSkyrim/steam_review_*.csv"))))
steam_HalfOfTheGameReviews = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/steam_review_*.csv"))))


C:\Users\Pyae Pyae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Pyae Pyae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Pyae Pyae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
steam_HalfOfTheGameReviews.head()

,Unnamed: 0,recommendationid,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,...,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,timestamp_dev_responded,developer_response
0,0,77156977,english,turd,1602038643,1602038643,True,0,0,0.0,...,True,76561199039769834,12,3,651.0,91.0,586.0,1.602042e+09,NaN,NaN
1,1,77156585,english,this is a great game and I hope more people wi...,1602037875,1602037875,True,0,0,0.0,...,True,76561198365076993,28,5,5269.0,1359.0,5269.0,1.601590e+09,NaN,NaN
2,2,77156574,english,Good game,1602037848,1602037848,True,0,0,0.0,...,True,76561198067756444,388,14,7993.0,509.0,7993.0,1.601969e+09,NaN,NaN
3,3,77155220,english,Haha bat hit zombie and zombie head explode,1602035382,1602035382,True,0,0,0.0,...,True,76561198168612142,66,3,6884.0,1081.0,6794.0,1.602046e+09,NaN,NaN
4,4,77153527,english,ZOMBIE GO GRRR GUN GO BRRRRRR,1602032063,1602032063,True,0,0,0.0,...,True,76561198048006611,62,2,6504.0,59.0,6504.0,1.601544e+09,NaN,NaN
